In [1]:
from pathlib import Path
import re
import pandas as pd
import sys

In [2]:

# ----- USER CONFIG -----
# Directory where your 36 CSV files live
INPUT_DIR = Path(r'E:\Uni_PGT\counts-data')  # <- updated path
# Output filename for the combined CSV
OUTPUT_FILE = Path(r'E:\Uni_PGT\counts-data\combined_od_with_datew.csv')
# Pattern to match files (will match filenames containing YYYY_MM or YYYY-MM)
GLOB_PATTERN = '*_counts*.csv'
# ------------------------


def extract_year_month_from_name(fname: str):
    
    """Return (year, month) tuple if found in filename, else None."""
    # look for 4-digit year, separator (_ or -), 2-digit month
    m = re.search(r'(\d{4})[_-](\d{2})', fname)
    if not m:
        return None
    year, month = m.group(1), m.group(2)
    return year, month


def main(input_dir: Path, output_file: Path):
    files = sorted(input_dir.glob(GLOB_PATTERN))
    if not files:
        print(f'No files found matching pattern {GLOB_PATTERN} in {input_dir.resolve()}')
        sys.exit(1)

    dfs = []
    skipped = []

    for f in files:
        ym = extract_year_month_from_name(f.name)
        if ym is None:
            skipped.append(f.name)
            continue

        year, month = ym
        # read CSV
        try:
            df = pd.read_csv(f)
        except Exception as e:
            print(f'Failed to read {f.name}: {e}')
            skipped.append(f.name)
            continue

        # add date columns in two common formats:
        # 'year_month' = 'YYYY-MM' and 'month_year' = 'MM/YYYY' (user asked for m/y)
        df['year_month'] = f"{year}-{month}"
        df['month_year'] = f"{month}/{year}"

        dfs.append(df)

    if not dfs:
        print('No CSVs successfully read (maybe filename pattern is different). Files skipped:')
        print('\n'.join(skipped))
        sys.exit(1)

    combined = pd.concat(dfs, ignore_index=True, sort=False)

    # Optional: reorder so date columns are near the front
    cols = list(combined.columns)
    for col in ['year_month', 'month_year']:
        if col in cols:
            cols.insert(0, cols.pop(cols.index(col)))
    combined = combined[cols]

    # Save the combined CSV
    combined.to_csv(output_file, index=False)
    print(f'Combined {len(dfs)} files into {output_file} (total rows: {len(combined)})')
    if skipped:
        print('Skipped files (no YYYY_MM found or read error):')
        print('\n'.join(skipped))


if __name__ == '__main__':
    main(INPUT_DIR, OUTPUT_FILE)


Combined 36 files into E:\Uni_PGT\counts-data\combined_od_with_datew.csv (total rows: 268488)


In [3]:
df=pd.read_csv(r'E:\Uni_PGT\counts-data\combined_od_with_date.csv')
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268488 entries, 0 to 268487
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   month_year        268488 non-null  object 
 1   year_month        268488 non-null  object 
 2   start_station_id  268488 non-null  int64  
 3   end_station_id    268488 non-null  int64  
 4   hour              268488 non-null  float64
 5   trip_count        268488 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 12.3+ MB


,start_station_id,end_station_id,hour,trip_count
count,268488.000000,268488.000000,268488.000000,268488.000000
mean,930.490595,976.395388,13.844608,1.714162
std,662.431144,671.219020,4.854848,1.803095
min,171.000000,171.000000,0.000000,1.000000
25%,260.000000,262.000000,11.000000,1.000000
50%,1024.000000,1025.000000,14.000000,1.000000
75%,1729.000000,1737.000000,17.000000,2.000000
max,2268.000000,2268.000000,23.000000,84.000000


In [4]:
REPORT_FILE = Path(r'E:\Uni_PGT\counts-data\data_quality_report.txt')
# Ensure correct data types
df['start_station_id'] = pd.to_numeric(df['start_station_id'], errors='coerce').astype('Int64')
df['end_station_id'] = pd.to_numeric(df['end_station_id'], errors='coerce').astype('Int64')
df['hour'] = pd.to_numeric(df['hour'], errors='coerce')
df['trip_count'] = pd.to_numeric(df['trip_count'], errors='coerce').astype('Int64')

# ----- MISSING VALUES -----
missing_summary = df.isna().sum()
missing_rows = df[df.isna().any(axis=1)]

if not missing_rows.empty:
    print(f"Found {len(missing_rows):,} rows with missing values — will drop them.")
    df = df.dropna()

In [5]:

# ----- DUPLICATES -----
num_dupes = df.duplicated().sum()
if num_dupes > 0:
    print(f"Dropping {num_dupes:,} duplicate rows.")
    df = df.drop_duplicates()

In [6]:

# ----- VALUE VALIDATION -----
# Check valid range for hour (0–23 expected)
invalid_hours = df[~df['hour'].between(0, 24)]
if not invalid_hours.empty:
    print(f"Found {len(invalid_hours):,} rows with invalid hour values (outside 0–24). Fixing...")
    df = df[df['hour'].between(0, 24)]

In [7]:
# Check station ID ranges
df_s=pd.read_csv(r'E:\Uni_PGT\station_data.csv')
m=list(df_s['station_id'].unique())
w=df['start_station_id'].unique()
missing_ids = [s for s in w if s not in m]
print(missing_ids)
print(len(missing_ids))


[np.int64(171), np.int64(255), np.int64(257), np.int64(261), np.int64(266), np.int64(273), np.int64(275), np.int64(277), np.int64(284), np.int64(285), np.int64(290), np.int64(297), np.int64(340), np.int64(341), np.int64(342), np.int64(343), np.int64(344), np.int64(345), np.int64(346), np.int64(347), np.int64(350), np.int64(351), np.int64(352), np.int64(353), np.int64(354), np.int64(355), np.int64(356), np.int64(357), np.int64(359), np.int64(365), np.int64(366), np.int64(648), np.int64(820), np.int64(860), np.int64(862), np.int64(863), np.int64(864), np.int64(865), np.int64(866), np.int64(867), np.int64(868), np.int64(869), np.int64(870), np.int64(871), np.int64(872), np.int64(873), np.int64(874), np.int64(875), np.int64(876), np.int64(877), np.int64(878), np.int64(879), np.int64(880), np.int64(881), np.int64(882), np.int64(885), np.int64(887), np.int64(888), np.int64(889), np.int64(883), np.int64(884), np.int64(890), np.int64(891), np.int64(901), np.int64(964), np.int64(965), np.int64(

In [8]:

# ----- REPORT -----
with open(REPORT_FILE, 'w', encoding='utf-8') as f:
    f.write('OD Matrix Data Quality Report\n')
    f.write('=' * 40 + '\n\n')
    f.write(f'Total rows after cleaning: {len(df):,}\n')
    f.write(f'Duplicates removed: {num_dupes}\n')
    f.write(f'Missing rows removed: {len(missing_rows)}\n')
    f.write(f'Invalid hour rows removed: {len(invalid_hours)}\n\n')


    f.write('Trip count summary (post-clean):\n')
    f.write(str(df['trip_count'].describe()) + '\n\n')


print(f'Data quality report saved to: {REPORT_FILE}')



Data quality report saved to: E:\Uni_PGT\counts-data\data_quality_report.txt


In [9]:
"""
combine_36_csvs_no_date.py

Reads all CSV files from cyclehire-cleandata named like 2018_10.csv ... 2021_09.csv
(or containing that yyyy_mm pattern in the filename), concatenates them into a single CSV
and saves it to cyclehire-cleandata\combined_all_periods.csv

Notes:
 - This script does NOT add a date column (as requested).
 - It will align columns by name; missing columns in some files will be filled with NaN.
 - It prints a short summary of files read and total rows combined.

Usage:
    python combine_36_csvs_no_date.py
"""

from pathlib import Path
import pandas as pd
import re

# ----- USER CONFIG -----
INPUT_DIR = Path(r'E:\Uni_PGT\cyclehire-cleandata')
OUTPUT_FILE = INPUT_DIR / 'combined_all_periods.csv'
GLOB_PATTERN = '*_*.csv'  # matches files with yyyy_mm in name like 2018_10.csv
FILE_FILTER_REGEX = re.compile(r'(20\d{2})[_-](0[1-9]|1[0-2])')  # restrict to yyyy_mm patterns
# ------------------------

# collect candidate files
files = sorted(INPUT_DIR.glob(GLOB_PATTERN))
selected_files = [f for f in files if FILE_FILTER_REGEX.search(f.name)]

if not selected_files:
    raise FileNotFoundError(f'No files matching yyyy_mm pattern found in {INPUT_DIR}')

print(f'Found {len(selected_files)} files to combine:')
for f in selected_files:
    print(' -', f.name)

# read and concatenate
dfs = []
for f in selected_files:
    try:
        df = pd.read_csv(f)
        df['__source_file'] = f.name  # optional: keep which file the row came from
        dfs.append(df)
    except Exception as e:
        print(f'Warning: failed to read {f.name}: {e}')

if not dfs:
    raise ValueError('No files were successfully read.')

combined = pd.concat(dfs, ignore_index=True, sort=False)
combined.to_csv(OUTPUT_FILE, index=False)

print(f'Combined {len(dfs)} files into {OUTPUT_FILE} (total rows: {len(combined):,})')

# optional quick sanity print
print('\nColumn summary (name : non-null count):')
print(combined.notna().sum().sort_values(ascending=False).head(50))

print('\nDone.')


Found 36 files to combine:
 - 2018_10.csv
 - 2018_11.csv
 - 2018_12.csv
 - 2019_01.csv
 - 2019_02.csv
 - 2019_03.csv
 - 2019_04.csv
 - 2019_05.csv
 - 2019_06.csv
 - 2019_07.csv
 - 2019_08.csv
 - 2019_09.csv
 - 2019_10.csv
 - 2019_11.csv
 - 2019_12.csv
 - 2020_01.csv
 - 2020_02.csv
 - 2020_03.csv
 - 2020_04.csv
 - 2020_05.csv
 - 2020_06.csv
 - 2020_07.csv
 - 2020_08.csv
 - 2020_09.csv
 - 2020_10.csv
 - 2020_11.csv
 - 2020_12.csv
 - 2021_01.csv
 - 2021_02.csv
 - 2021_03.csv
 - 2021_04.csv
 - 2021_05.csv
 - 2021_06.csv
 - 2021_07.csv
 - 2021_08.csv
 - 2021_09.csv


<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\frequensy\AppData\Local\Temp\ipykernel_21108\2259740896.py:1: SyntaxWarning: invalid escape sequence '\c'
  """


Combined 36 files into E:\Uni_PGT\cyclehire-cleandata\combined_all_periods.csv (total rows: 460,655)

Column summary (name : non-null count):
started_at                   460655
ended_at                     460655
duration                     460655
start_station_id             460655
start_station_name           460655
start_station_latitude       460655
end_station_latitude         460655
start_station_longitude      460655
end_station_id               460655
end_station_name             460655
__source_file                460655
end_station_longitude        460655
start_station_description    456167
end_station_description      455560
dtype: int64

Done.


In [10]:
print(combined.info())
print(combined.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460655 entries, 0 to 460654
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   started_at                 460655 non-null  object 
 1   ended_at                   460655 non-null  object 
 2   duration                   460655 non-null  int64  
 3   start_station_id           460655 non-null  int64  
 4   start_station_name         460655 non-null  object 
 5   start_station_description  456167 non-null  object 
 6   start_station_latitude     460655 non-null  float64
 7   start_station_longitude    460655 non-null  float64
 8   end_station_id             460655 non-null  int64  
 9   end_station_name           460655 non-null  object 
 10  end_station_description    455560 non-null  object 
 11  end_station_latitude       460655 non-null  float64
 12  end_station_longitude      460655 non-null  float64
 13  __source_file              46

In [11]:
# contiguous_hour_clustering_and_gravity_avg_per_day.py
"""
Same clustering + gravity script you provided, but changes to produce
OD aggregated per cluster *averaged per calendar day*.

Key change: after summing hourly OD matrices for a cluster we divide
by `n_days` (the number of unique calendar dates present in the
`combined` DataFrame). This turns cluster totals (total trips across
all days in the dataset occurring during those cluster hours) into
an average number of trips per calendar day for that cluster.

Notes / caveats:
 - `n_days` is computed globally from `combined['started_at'].dt.date.nunique()`.
   This is the simplest and most defensible choice: it gives average trips
   per calendar day across the whole observation window. If you prefer to
   compute `n_days` per-hour or per-cluster (e.g. count of days that actually
   contained at least one trip in the cluster hours), see the comment below
   and I can provide that variant.
 - The averaged `agg` (OD) is then used to compute O and D (origin/destination
   totals) and run the gravity model. The gravity model will therefore predict
   average trips per calendar day for that cluster period.

Run this in the same session where `combined` (the concatenated trip DataFrame)
exists in memory.
"""

from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from scipy.sparse import csr_matrix

# ---------- USER CONFIG ----------
OUTPUT_DIR = Path(r'E:/Uni_PGT/visualisation_outputs/clustered_gravity_avg_per_day')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
CLUSTER_COUNT = 4            # number of contiguous clusters to form
PCA_VARIANCE = 0.90          # keep PCA components explaining this fraction of variance
BETA_1 = 0.0005
ERROR_THRESHOLD = 0.01
IMPROVEMENT_THRESHOLD = 1e-6
MAX_ITERS = 2000
EPS = 1e-12
# ---------------------------------

# Ensure combined exists
try:
    combined
except NameError:
    raise RuntimeError('DataFrame `combined` not found in memory. Load it first as `combined`.')

# Prepare datetime and hour
combined['started_at'] = pd.to_datetime(combined['started_at'], errors='coerce')
combined = combined.dropna(subset=['started_at'])
if 'hour' not in combined.columns:
    combined['hour'] = combined['started_at'].dt.hour

# Compute number of calendar days present in the dataset (global)
# This is used to convert cluster totals -> average trips per calendar day
n_days = combined['started_at'].dt.date.nunique()
print(f"Dataset spans {n_days} calendar days (unique dates) — cluster OD will be averaged per calendar day.")

Dataset spans 1083 calendar days (unique dates) — cluster OD will be averaged per calendar day.


In [12]:

# Optional alternative (commented): compute n_days_per_cluster by counting unique dates
# with at least one trip in cluster hours. This can yield slightly different averages
# that only count days where cluster hours had any activity. If you prefer that,
# uncomment and use the per-cluster approach shown later in a comment.

# Precompute hourly OD pivot tables (raw counts per hour)
hourly_pivots = {}
for h in range(24):
    sub = combined[combined['hour'] == h]
    if sub.empty:
        hourly_pivots[h] = pd.DataFrame()
        continue
    counts = sub.groupby(['start_station_id', 'end_station_id']).size().reset_index(name='count')
    pivot = counts.pivot(index='start_station_id', columns='end_station_id', values='count').fillna(0)
    hourly_pivots[h] = pivot

# Build aligned feature vectors for each hour using union of station ids
hours = sorted(hourly_pivots.keys())
all_stations = sorted({int(s) for h in hours for s in (list(hourly_pivots[h].index) + list(hourly_pivots[h].columns))})
if len(all_stations) == 0:
    raise RuntimeError('No station IDs found in hourly pivots--check combined data')

def pivot_to_aligned_vector(pivot_df, stations):
    mat = pd.DataFrame(0.0, index=stations, columns=stations)
    if not pivot_df.empty:
        tmp = pivot_df.reindex(index=stations, columns=stations).fillna(0)
        mat.iloc[:, :] = tmp.values
    return mat.values.flatten()

X_list = []
for h in hours:
    vec = pivot_to_aligned_vector(hourly_pivots[h], all_stations)
    X_list.append(vec)
X = np.vstack(X_list)  # shape (24, n_features)

# Scale + PCA
scaler = StandardScaler(with_mean=True, with_std=True)
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=PCA_VARIANCE, svd_solver='full')
X_pca = pca.fit_transform(X_scaled)

# Build chain connectivity (adjacency) for contiguous clustering
n_hours = X_pca.shape[0]
rows = []
cols = []
data = []
for i in range(n_hours - 1):
    rows.extend([i, i+1])
    cols.extend([i+1, i])
    data.extend([1, 1])
connectivity = csr_matrix((data, (rows, cols)), shape=(n_hours, n_hours))

# Run contiguous agglomerative clustering
agg = AgglomerativeClustering(n_clusters=CLUSTER_COUNT, linkage='ward', connectivity=connectivity)
labels = agg.fit_predict(X_pca)

hour_cluster_df = pd.DataFrame({'hour': hours, 'cluster': labels}).sort_values('hour')
hour_cluster_df.to_csv(OUTPUT_DIR / 'cluster_membership.csv', index=False)
print('Hour -> Cluster mapping saved to', OUTPUT_DIR / 'cluster_membership.csv')

Hour -> Cluster mapping saved to E:\Uni_PGT\visualisation_outputs\clustered_gravity_avg_per_day\cluster_membership.csv


In [13]:

# Helper: haversine pairwise
def haversine_pairwise(lons, lats):
    R = 6371000.0
    lon = np.radians(lons)
    lat = np.radians(lats)
    dlon = lon[:, None] - lon[None, :]
    dlat = lat[:, None] - lat[None, :]
    a = np.sin(dlat/2.0)**2 + np.cos(lat[:, None]) * np.cos(lat[None, :]) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
    return R * c

# Impedance function (single det1 used here)
def new_cost1(cost_matrix, beta=BETA_1):
    return np.exp(-beta * cost_matrix)

# Gravity model (doubly-constrained IPF)
def gravity_model(O, D, det, error_threshold=ERROR_THRESHOLD, improvement_threshold=IMPROVEMENT_THRESHOLD, max_iters=MAX_ITERS):
    O = np.array(O, dtype=float).copy()
    D = np.array(D, dtype=float).copy()
    sum_O = O.sum(); sum_D = D.sum()
    if sum_O <= 0 or sum_D <= 0:
        raise ValueError('Origin or Destination totals sum to zero')
    if abs(sum_O - sum_D) > 1e-9:
        D = D * (sum_O / sum_D)
    n = len(O)
    Ai = np.ones(n)
    Bj = np.ones(n)
    prev_error = np.inf
    Tij = np.zeros((n, n), dtype=float)
    det_mat = np.array(det, dtype=float).copy()
    det_mat[det_mat < EPS] = EPS
    iteration = 0
    while iteration < max_iters:
        iteration += 1
        denom_i = (det_mat * (Bj * D)[None, :]).sum(axis=1) + EPS
        Ai = 1.0 / denom_i
        denom_j = (det_mat * (Ai * O)[:, None]).sum(axis=0) + EPS
        Bj_new = 1.0 / denom_j
        Tij = (Ai * O)[:, None] * (Bj_new * D)[None, :] * det_mat
        error = (np.abs(O - Tij.sum(axis=1)).sum() + np.abs(D - Tij.sum(axis=0)).sum()) / (sum_O + EPS)
        improvement = abs(prev_error - error)
        if error < error_threshold:
            stop_reason = 'Error threshold met'
            break
        if improvement < improvement_threshold:
            stop_reason = 'Slow improvement'
            break
        prev_error = error
        Bj = Bj_new
    else:
        stop_reason = 'max_iters'
    diagnostics = {'iterations': iteration, 'error': float(error), 'stop_reason': stop_reason}
    return Tij, diagnostics

# Metrics
def calculate_metrics(predicted_T, observed_T_df):
    obs = observed_T_df.to_numpy().astype(float)
    pred = np.array(predicted_T, dtype=float)
    if obs.shape != pred.shape:
        raise ValueError('Predicted and observed shapes differ')
    obs_f = obs.flatten(); pred_f = pred.flatten()
    mse = np.mean((obs_f - pred_f) ** 2)
    rmse = float(np.sqrt(mse))
    ss_tot = np.sum((obs_f - obs_f.mean()) ** 2)
    ss_res = np.sum((obs_f - pred_f) ** 2)
    r2 = float(1.0 - (ss_res / (ss_tot + EPS)))
    return {'rmse': rmse, 'r2': r2}

# Aggregate OD per cluster and run gravity — now averaging per calendar day
for c in sorted(hour_cluster_df['cluster'].unique()):
    hrs = hour_cluster_df[hour_cluster_df['cluster'] == c]['hour'].tolist()
    print(f'Processing cluster {c}: hours = {hrs}')
    # sum OD counts across hours in cluster
    agg = None
    for h in hrs:
        p = hourly_pivots[h]
        if agg is None:
            agg = p.copy()
        else:
            agg = agg.add(p, fill_value=0)
    agg = agg.fillna(0)

    # ----- NEW: convert cluster totals -> average per calendar day -----
    # Divide the aggregated matrix by the number of calendar days in the dataset
    # so that values represent "average trips per calendar day during the cluster hours".
    if n_days > 0:
        agg = agg / float(n_days)
    else:
        print('Warning: n_days==0, skipping division (no date information)')
    # ------------------------------------------------------------------

    if agg.empty:
        print(f'Cluster {c}: empty aggregated OD, skipping')
        continue

    # ensure square by intersection
    rows = [int(x) for x in agg.index.tolist()]
    cols = [int(x) for x in agg.columns.tolist()]
    common = sorted(list(set(rows) & set(cols)))
    if len(common) < 2:
        print(f'Cluster {c}: too few common stations ({len(common)}), skipping')
        continue
    agg = agg.reindex(index=common, columns=common).fillna(0)

    # Save the averaged cluster OD (this CSV now contains average trips per calendar day for the cluster hours)
    agg.to_csv(OUTPUT_DIR / f'cluster_od_avgperday_{c}.csv')

    n = len(common)
    print(f'Cluster {c}: n_stations={n} (averaged over {n_days} days)')
    if n > 1500:
        print('WARNING: cluster has many stations (>1500); this may be slow and memory-heavy')

    # build coord df (median of start/end coords)
    lon_cols = [col for col in combined.columns if 'longitude' in col.lower()]
    lat_cols = [col for col in combined.columns if 'latitude' in col.lower()]
    start_lon = next((col for col in lon_cols if col.lower().startswith('start')), None)
    start_lat = next((col for col in lat_cols if col.lower().startswith('start')), None)
    end_lon = next((col for col in lon_cols if col.lower().startswith('end')), None)
    end_lat = next((col for col in lat_cols if col.lower().startswith('end')), None)
    coord_parts = []
    if start_lon and start_lat:
        tmp = combined.groupby('start_station_id')[[start_lon, start_lat]].median()
        tmp = tmp.rename(columns={start_lon:'lon', start_lat:'lat'})
        coord_parts.append(tmp)
    if end_lon and end_lat:
        tmp2 = combined.groupby('end_station_id')[[end_lon, end_lat]].median()
        tmp2 = tmp2.rename(columns={end_lon:'lon', end_lat:'lat'})
        coord_parts.append(tmp2)
    if not coord_parts:
        raise RuntimeError('Could not find station lon/lat columns in combined')
    coord_df = pd.concat(coord_parts).groupby(level=0).first()
    coord_df = coord_df.reindex(common).dropna()
    if len(coord_df) != n:
        missing = set(common) - set(coord_df.index.astype(int).tolist())
        if missing:
            print(f'Cluster {c}: dropping {len(missing)} stations with missing coords (sample): {list(missing)[:10]}')
            keep = [s for s in common if s not in missing]
            if len(keep) < 2:
                print(f'Cluster {c}: too few stations after dropping, skipping')
                continue
            agg = agg.reindex(index=keep, columns=keep).fillna(0)
            coord_df = coord_df.reindex(keep)
            common = keep
            n = len(common)

    # compute cost matrix
    lons = coord_df['lon'].to_numpy(dtype=float)
    lats = coord_df['lat'].to_numpy(dtype=float)
    cost_m = haversine_pairwise(lons, lats)
    pd.DataFrame(cost_m, index=common, columns=common).to_csv(OUTPUT_DIR / f'cost_matrix_cluster_{c}.csv')

    # deterrence matrix
    det1 = new_cost1(cost_m, beta=BETA_1)

    # totals (these totals are now average trips per calendar day for the cluster hours)
    O = agg.sum(axis=1).to_numpy()
    D = agg.sum(axis=0).to_numpy()

    # run gravity
    Tij1, diag1 = gravity_model(O.copy(), D.copy(), det1)
    pred1_df = pd.DataFrame(Tij1, index=agg.index, columns=agg.columns)
    pred1_df.to_csv(OUTPUT_DIR / f'predicted_gravity_det1_cluster_{c}_avgperday.csv')
    metrics1 = calculate_metrics(pred1_df, agg)

    pd.DataFrame([diag1]).to_csv(OUTPUT_DIR / f'diagnostics_det1_cluster_{c}.csv', index=False)
    pd.DataFrame([metrics1]).to_csv(OUTPUT_DIR / f'metrics_det1_cluster_{c}.csv', index=False)

    print(f'Cluster {c}: done. metrics_det1={metrics1}')

print('All clusters processed. Outputs in', OUTPUT_DIR)


Processing cluster 0: hours = [11, 12, 13, 14, 15]
Cluster 0: n_stations=191 (averaged over 1083 days)
Cluster 0: done. metrics_det1={'rmse': 0.0165686802815451, 'r2': 0.5223031333344694}
Processing cluster 1: hours = [16, 17]
Cluster 1: n_stations=186 (averaged over 1083 days)
Cluster 1: done. metrics_det1={'rmse': 0.006940969466649425, 'r2': 0.5754631478891745}
Processing cluster 2: hours = [18, 19, 20, 21, 22, 23]
Cluster 2: n_stations=182 (averaged over 1083 days)
Cluster 2: done. metrics_det1={'rmse': 0.008103426581723673, 'r2': 0.6272312037982717}
Processing cluster 3: hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Cluster 3: n_stations=187 (averaged over 1083 days)
Cluster 3: done. metrics_det1={'rmse': 0.00854909312389969, 'r2': 0.5308287108154313}
All clusters processed. Outputs in E:\Uni_PGT\visualisation_outputs\clustered_gravity_avg_per_day


In [14]:
# beta_grid_det1_avg_per_day.py
# Updated beta-grid search script that expects cluster-aggregated OD to be
# averaged per calendar day before running gravity model.
# This version computes n_days from the `combined` DataFrame and divides the
# aggregated cluster OD by n_days (so O/D represent average trips per calendar day).

import numpy as np
import pandas as pd
from pathlib import Path

# ---------- USER / RUN-TIME CONFIG ----------
beta_grid = np.logspace(-6, -2, 25)   # grid to search for beta (adjust if desired)
OUTPUT_DIR = Path(r'E:/Uni_PGT/visualisation_outputs/clustered_gravity')  # match your cluster script
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
EPS = 1e-12
# --------------------------------------------

# Basic haversine (meters)
def haversine_pairwise(lons, lats):
    R = 6371000.0
    lon = np.radians(lons)
    lat = np.radians(lats)
    dlon = lon[:, None] - lon[None, :]
    dlat = lat[:, None] - lat[None, :]
    a = np.sin(dlat/2.0)**2 + np.cos(lat[:, None]) * np.cos(lat[None, :]) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
    return R * c

# Normalized RMSE helper
def normalized_rmse(pred_df, obs_df):
    obs = obs_df.to_numpy(dtype=float)
    pred = pred_df.to_numpy(dtype=float)
    mse = np.mean((obs - pred) ** 2)
    rmse = float(np.sqrt(mse))
    mean_obs = float(np.mean(obs))
    if mean_obs == 0:
        return {'rmse': rmse, 'nrmse': float('inf')}
    return {'rmse': rmse, 'nrmse': float(rmse / mean_obs)}

# Ensure gravity_model & calculate_metrics exist (from your previous script). If not, provide safe fallback.
try:
    gravity_model  # noqa
except NameError:
    raise RuntimeError("gravity_model is not defined in the session. Run the clustering/gravity script first (which defines gravity_model).")

try:
    calculate_metrics  # noqa
except NameError:
    # fallback basic metrics (should be similar to your earlier calculate_metrics)
    def calculate_metrics(pred_df, obs_df):
        obs = obs_df.to_numpy(dtype=float)
        pred = pred_df.to_numpy(dtype=float)
        if obs.shape != pred.shape:
            raise ValueError('Predicted and observed shapes differ')
        obs_f = obs.flatten()
        pred_f = pred.flatten()
        mse = np.mean((obs_f - pred_f) ** 2)
        rmse = float(np.sqrt(mse))
        ss_total = np.sum((obs_f - obs_f.mean()) ** 2)
        ss_residual = np.sum((obs_f - pred_f) ** 2)
        r_squared = float(1.0 - (ss_residual / (ss_total + EPS)))
        return {'rmse': rmse, 'r2': r_squared}

# Core function: runs grid search for det1 on a single aggregated cluster
def run_beta_grid_on_cluster(agg_df, coord_df, cost_func, det_func_factory, beta_values, gravity_model_func, metrics_func):
    """
    Returns best = {'beta', 'metrics', 'pred_df', 'diag'} using primary=r2, secondary=-nrmse
    """
    common = [int(x) for x in agg_df.index.tolist()]
    lons = coord_df.loc[common, 'lon'].to_numpy(dtype=float)
    lats = coord_df.loc[common, 'lat'].to_numpy(dtype=float)
    cost_m = cost_func(lons, lats)

    O = agg_df.sum(axis=1).to_numpy()
    D = agg_df.sum(axis=0).to_numpy()

    best = {'beta': None, 'metrics': None, 'pred_df': None, 'diag': None}
    best_score = None

    for beta in beta_values:
        det = det_func_factory(cost_m, beta=beta)
        try:
            Tij, diag = gravity_model_func(O.copy(), D.copy(), det)
        except Exception:
            # solver failure for this beta — skip
            continue
        pred_df = pd.DataFrame(Tij, index=agg_df.index, columns=agg_df.columns)
        mets = metrics_func(pred_df, agg_df)    # should include 'r2'
        nr = normalized_rmse(pred_df, agg_df)
        mets.update(nr)
        score = (mets.get('r2', -9999), - (mets.get('nrmse', np.inf) if np.isfinite(mets.get('nrmse', np.inf)) else np.inf))
        if best_score is None or score > best_score:
            best_score = score
            best = {'beta': float(beta), 'metrics': mets, 'pred_df': pred_df.copy(), 'diag': diag}
    return best

# DET1 factory (exponential), no det2 per your request
def det1_factory(costm, beta):
    return np.exp(-beta * costm)

# Containers for results
best_betas = {}          # {cluster: {'beta':..., 'metrics':..., 'diag':...}}
predicted_matrices = {}  # {cluster: predicted_df}

# Compute number of calendar days in the combined dataset (used to convert cluster totals -> avg per calendar day)
try:
    n_days = int(combined['started_at'].dt.date.nunique())
except Exception:
    # fallback: if combined not present or started_at not parsed, set to 1 (no scaling)
    n_days = 1

clusters = sorted(hour_cluster_df['cluster'].unique())

for c in clusters:
    hrs = hour_cluster_df[hour_cluster_df['cluster']==c]['hour'].tolist()
    # aggregate OD across hours in this cluster
    agg = None
    for h in hrs:
        p = hourly_pivots[h]
        if agg is None:
            agg = p.copy()
        else:
            agg = agg.add(p, fill_value=0)
    if agg is None:
        print(f'Cluster {c}: empty aggregation, skipping')
        continue
    agg = agg.fillna(0)

    # --- NEW: convert aggregated cluster totals to average trips per calendar day ---
    if n_days > 1:
        agg = agg / float(n_days)
    # save the averaged cluster OD for diagnostic purposes
    agg.to_csv(OUTPUT_DIR / f'cluster_od_avg_per_day_{c}.csv')

    # square matrix by intersection of rows & cols
    rows = [int(x) for x in agg.index.tolist()]
    cols = [int(x) for x in agg.columns.tolist()]
    common = sorted(list(set(rows) & set(cols)))
    if len(common) < 2:
        print(f'Cluster {c} skipped (too few common stations)')
        continue
    agg = agg.reindex(index=common, columns=common).fillna(0)

    # build coordinate DF for these common stations (median of start/end coords)
    lon_cols = [col for col in combined.columns if 'longitude' in col.lower()]
    lat_cols = [col for col in combined.columns if 'latitude' in col.lower()]
    start_lon = next((col for col in lon_cols if col.lower().startswith('start')), None)
    start_lat = next((col for col in lat_cols if col.lower().startswith('start')), None)
    end_lon = next((col for col in lon_cols if col.lower().startswith('end')), None)
    end_lat = next((col for col in lat_cols if col.lower().startswith('end')), None)
    coord_parts = []
    if start_lon and start_lat:
        tmp = combined.groupby('start_station_id')[[start_lon, start_lat]].median()
        tmp = tmp.rename(columns={start_lon:'lon', start_lat:'lat'})
        coord_parts.append(tmp)
    if end_lon and end_lat:
        tmp2 = combined.groupby('end_station_id')[[end_lon, end_lat]].median()
        tmp2 = tmp2.rename(columns={end_lon:'lon', end_lat:'lat'})
        coord_parts.append(tmp2)
    if not coord_parts:
        raise RuntimeError('Could not find station longitude/latitude columns in `combined`.')

    coord_df = pd.concat(coord_parts).groupby(level=0).first()
    coord_df = coord_df.reindex(common).dropna()

    if len(coord_df) != len(common):
        missing = set(common) - set(coord_df.index.astype(int).tolist())
        print(f'Cluster {c}: dropping {len(missing)} stations missing coords (sample): {list(missing)[:6]}')
        keep = [s for s in common if s not in missing]
        agg = agg.reindex(index=keep, columns=keep).fillna(0)
        coord_df = coord_df.reindex(keep)
        common = keep
        if len(common) < 2:
            print(f'Cluster {c}: too few stations after dropping coords, skipping')
            continue

    # run grid search for det1 only
    best1 = run_beta_grid_on_cluster(agg, coord_df, haversine_pairwise, det1_factory, beta_grid, gravity_model, calculate_metrics)

    # save best1 outputs
    if best1['beta'] is None:
        print(f'Cluster {c}: no successful beta found (solver failed for all candidates)')
        continue

    best_betas[c] = {'beta_det1': best1['beta'], 'metrics_det1': best1['metrics'], 'diag_det1': best1['diag']}
    predicted_matrices[c] = best1['pred_df']

    # persist to disk
    best1['pred_df'].to_csv(OUTPUT_DIR / f'best_pred_det1_clusterr_{c}.csv')
    pd.DataFrame([best1['metrics']]).to_csv(OUTPUT_DIR / f'best_metrics_det1_clusterr_{c}.csv', index=False)

    print(f"Cluster {c}: done. best_beta_det1={best1['beta']:.2e}, metrics={best1['metrics']}")

# save summary CSV
summary = []
for c, info in best_betas.items():
    summary.append({'cluster': c, 'best_beta_det1': info['beta_det1'], **{f"metrics_{k}": v for k,v in info['metrics_det1'].items()}})
summary_df = pd.DataFrame(summary)
summary_df.to_csv(OUTPUT_DIR / 'beta_gridsearch_summary_det1_only.csv', index=False)

print('Grid search (det1 only) completed. Summary saved to:', OUTPUT_DIR / 'beta_gridsearch_summary_det1_only.csv')
print('Best betas dict: variable `best_betas` (in memory).')
print('Predicted OD DataFrames per cluster: variable `predicted_matrices` (in memory).')

Cluster 0: done. best_beta_det1=3.16e-04, metrics={'rmse': 0.01511597184838509, 'r2': 0.6023978664038023, 'nrmse': 3.3690183047494133}
Cluster 1: done. best_beta_det1=4.64e-04, metrics={'rmse': 0.0068731200005717336, 'r2': 0.583722459017577, 'nrmse': 3.1473002820950526}
Cluster 2: done. best_beta_det1=6.81e-04, metrics={'rmse': 0.007998473455017589, 'r2': 0.6368246502400101, 'nrmse': 2.905371397098946}
Cluster 3: done. best_beta_det1=4.64e-04, metrics={'rmse': 0.0085127296413361, 'r2': 0.5348114533084086, 'nrmse': 3.1500856842441234}
Grid search (det1 only) completed. Summary saved to: E:\Uni_PGT\visualisation_outputs\clustered_gravity\beta_gridsearch_summary_det1_only.csv
Best betas dict: variable `best_betas` (in memory).
Predicted OD DataFrames per cluster: variable `predicted_matrices` (in memory).


In [15]:
"""
station_to_poi_od_from_predicted_fixed.py

Fixed version of the POI-OD conversion that uses gravity-model predicted
station->station OD matrices (per cluster), and avoids the ValueError you ran
into by correctly sizing the POI index for each output matrix.

Assumptions:
 - `predicted_matrices` is a dict in memory mapping cluster -> pandas.DataFrame
   with station IDs as index and columns (square) containing predicted Tij counts.
 - `combined` DataFrame is in memory (for station coords).
 - POIs CSV is at POI_FILE and contains columns ['lat','lon','category'] and
   optionally 'poi_id'.

Saves per-cluster POI OD CSVs (only for POIs that receive >0 allocation for
stations in that cluster), plus an aggregated POI OD across clusters.
"""

from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors


In [16]:
# --- load POIs ---
POI_FILE = r'E:\Uni_PGT\Express\edinburgh_pois.csv'
OUTPUT_DIR = Path(r'E:\Uni_PGT\visualisation_outputs\poi_od_fixed')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

K_NEIGHBORS = 3
CLOSE_RADIUS_METERS = 500
CATEGORY_WEIGHTS_uniform = {
    'library': 1,
    'school': 1,
    'university': 1,
    'residential': 1,
    'commercial': 1,
    'hospital': 1
}


In [17]:
# Create a mapping matrix (these are allocation percentages)
TRAVEL_TO_POI_MAPPING = {
    'Commuting (Travel)': {
        'residential': 0.40,    # Trip origins from home
        'commercial': 0.35,     # Work destinations
        'university': 0.15,     # Student commuting
        'school': 0.10      # Government jobs
    },
    'Business (Travel)': {
        'commercial': 0.90,     # Business meetings, offices
        'hospital': 0.10        # Business at medical facilities
    },
    'Education/escort education (Travel)': {
        'school': 0.4,         # Primary/secondary schools
        'university': 0.6     # Higher education
    },
    'Shopping (Leisure)': {
        'commercial': 1.00      # All shopping trips
    },
    'Other escort (Travel)': {
        'school': 0.30,         # Dropping off kids
        'hospital': 0.30,       # Medical appointments
        'commercial': 0.20,     # Errands while escorting
        'residential': 0.20    # To/from homes
    },
    'Personal business (Travel)': {
        'commercial': 0.35,     # Banks, services
        'hospital': 0.35,       # Medical
        'library': 0.20,        # Community services
        'residential': 0.10    # To/from homes

    },
    'Leisure': {
        'commercial': 0.75,     # Restaurants, entertainment
        'library': 0.10,        # Libraries, museums
        'residential': 0.15 
    }
}

In [18]:
# Use your actual travel percentages
travel_data = {
    'Commuting (Travel)': 0.28,
    'Business (Travel)': 0.02,
    'Education/escort education (Travel)': 0.11,
    'Shopping (Leisure)': 0.10,
    'Other escort (Travel)': 0.02,
    'Personal business (Travel)': 0.06,
    'Leisure': 0.43
}

# Calculate final weights
poi_weights = {}
for travel_purpose, travel_pct in travel_data.items():
    allocation = TRAVEL_TO_POI_MAPPING[travel_purpose]
    for poi_category, alloc_pct in allocation.items():
        poi_weights[poi_category] = poi_weights.get(poi_category, 0) + (travel_pct * alloc_pct)

# Normalize
total = sum(poi_weights.values())
category_weights_pred = {k: v/total for k, v in poi_weights.items()}
print(category_weights_pred)

{'residential': 0.18284313725490198, 'commercial': 0.5524509803921569, 'university': 0.10588235294117648, 'school': 0.07647058823529412, 'hospital': 0.028431372549019604, 'library': 0.05392156862745098}


In [19]:
# --- DBSCAN CLUSTERING ---
from sklearn.cluster import DBSCAN
# Commercial-focused weights (commercial POIs weighted higher)
CATEGORY_WEIGHTS_COMMERCIAL = {
    'library': 1,
    'school': 1,
    'university': 1,
    'residential': 1,
    'commercial': 15,  # Commercial POIs get more weight
    'hospital': 1
}
def diff_weights(cat_weight):
    EPS = 1e-12
    if cat_weight['commercial']==cat_weight['library']:
        t='uniform'
    elif cat_weight['commercial']<15*cat_weight['library']:
        t='pred'
    else:
        t='commercial'
    pois = pd.read_csv(POI_FILE)
    pois = pois.dropna(subset=['lat', 'lon']).reset_index(drop=True)
    if 'poi_id' not in pois.columns:
        pois['poi_id'] = pois.index.astype(int)
    pois['category'] = pois['category'].astype(str).fillna('commercial').str.lower().str.strip()
    pois['weight'] = pois['category'].map(cat_weight).fillna(1.0)
    
    
    
    print("DBSCAN clustering...")
    
    original_pois = pois.copy()
    clustered_pois_list = []
    current_poi_id = 0
    
    # Process in smaller batches by category to reduce memory usage
    for category in pois['category'].unique():
        category_pois = pois[pois['category'] == category].copy().reset_index(drop=True)
        
        if len(category_pois) <= 1:
            # Single POI in this category - keep as is
            for _, poi in category_pois.iterrows():
                clustered_pois_list.append({
                    'poi_id': current_poi_id,
                    'lat': poi['lat'],
                    'lon': poi['lon'],
                    'category': category,
                    'weight': poi['weight'],
                    'original_poi_count': 1,
                    'original_poi_ids': [poi['poi_id']]
                })
                current_poi_id += 1
            continue
        
        print(f"  Processing {category}: {len(category_pois)} POIs")
        
        # Convert to radians for haversine distance
        coords_rad = np.radians(category_pois[['lat', 'lon']].values)
        
        # Use DBSCAN with optimized parameters
        # eps in radians: 200 meters / Earth radius
        eps_rad = 100 / 6371000
        
        # Use algorithm='ball_tree' for better memory efficiency with haversine
        dbscan = DBSCAN(
            eps=eps_rad,
            min_samples=1,  # Every point forms a cluster
            metric='haversine',
            algorithm='ball_tree',  # More memory efficient for haversine
            n_jobs=-1  # Use all available cores
        )
        
        # Fit and get labels
        labels = dbscan.fit_predict(coords_rad)
        
        # Create clusters from labels
        clusters = {}
        for i, label in enumerate(labels):
            if label not in clusters:
                clusters[label] = []
            clusters[label].append(i)
        
        # Create clustered POIs
        for label, indices in clusters.items():
            cluster_data = category_pois.iloc[indices]
            
            if len(cluster_data) == 1:
                poi = cluster_data.iloc[0]
                clustered_pois_list.append({
                    'poi_id': current_poi_id,
                    'lat': poi['lat'], 'lon': poi['lon'],
                    'category': category, 'weight': poi['weight'],
                    'original_poi_count': 1,
                    'original_poi_ids': [poi['poi_id']]
                })
            else:
                clustered_pois_list.append({
                    'poi_id': current_poi_id,
                    'lat': cluster_data['lat'].mean(),
                    'lon': cluster_data['lon'].mean(),
                    'category': category,
                    'weight': cluster_data['weight'].sum(),
                    'original_poi_count': len(cluster_data),
                    'original_poi_ids': cluster_data['poi_id'].tolist()
                })
            current_poi_id += 1
    
    pois = pd.DataFrame(clustered_pois_list)
    print(f"Reduced from {len(original_pois)} to {len(pois)} POI clusters")
    output=Path(r'E:\Uni_PGT\visualisation_outputs')
    pois.to_csv(output/ f'pois_{t}.csv')
    print('Saved reduced pois to', r'E:\Uni_PGT\visualisation_outputs')
    # --- build station coords (median of start/end) ---
    print(original_pois.head())
    print(pois.head())
    return pois
pois_uniform=diff_weights(CATEGORY_WEIGHTS_uniform)
pois_pred=diff_weights(category_weights_pred)
pois_commercial=diff_weights(CATEGORY_WEIGHTS_COMMERCIAL)

DBSCAN clustering...
  Processing library: 49 POIs
  Processing school: 188 POIs
  Processing university: 28 POIs
  Processing residential: 32837 POIs
  Processing commercial: 551 POIs
  Processing hospital: 16 POIs
Reduced from 33669 to 885 POI clusters
Saved reduced pois to E:\Uni_PGT\visualisation_outputs
                                name                       geometry  \
0              Wester Hailes Library  POINT (-3.2851455 55.9162285)   
1          Pirniehall Primary School  POINT (-3.2510739 55.9737056)   
2               Corstorphine Library  POINT (-3.2810362 55.9407099)   
3  Fettes College Preparatory School  POINT (-3.2259783 55.9666246)   
4                           Haywired  POINT (-3.1233362 55.9344588)   

         lat       lon category  poi_id  weight  
0  55.916229 -3.285146  library       0       1  
1  55.973706 -3.251074   school       1       1  
2  55.940710 -3.281036  library       2       1  
3  55.966625 -3.225978   school       3       1  
4  55.934459 

In [20]:

# --- build station coords (median of start/end) ---
lon_cols = [c for c in combined.columns if 'longitude' in c.lower()]
lat_cols = [c for c in combined.columns if 'latitude' in c.lower()]
start_lon = next((c for c in lon_cols if c.lower().startswith('start')), None)
start_lat = next((c for c in lat_cols if c.lower().startswith('start')), None)
end_lon = next((c for c in lon_cols if c.lower().startswith('end')), None)
end_lat = next((c for c in lat_cols if c.lower().startswith('end')), None)

coord_parts = []
if start_lon and start_lat:
    tmp = combined.groupby('start_station_id')[[start_lon, start_lat]].median()
    tmp = tmp.rename(columns={start_lon: 'lon', start_lat: 'lat'})
    coord_parts.append(tmp)
if end_lon and end_lat:
    tmp2 = combined.groupby('end_station_id')[[end_lon, end_lat]].median()
    tmp2 = tmp2.rename(columns={end_lon: 'lon', end_lat: 'lat'})
    coord_parts.append(tmp2)
if not coord_parts:
    raise RuntimeError('No station lon/lat columns found in `combined`.')

stations_df = pd.concat(coord_parts).groupby(level=0).first()
stations_df = stations_df.rename_axis('station_id')
stations_df.index = stations_df.index.astype(int)
# restrict to stations present in any predicted matrix to avoid waste
pred_station_ids = set()
for df in predicted_matrices.values():
    pred_station_ids.update([int(x) for x in df.index.astype(int).tolist()])
stations_df = stations_df.reindex(sorted(pred_station_ids)).dropna()
if stations_df.empty:
    raise RuntimeError('No station coordinates available for stations present in predicted_matrices.')
station_ids = stations_df.index.astype(int).tolist()

# --- helper: haversine (vectorized) ---
def haversine_matrix(lonA, latA, lonB, latB):
    R = 6371000.0
    lonA = np.radians(np.asarray(lonA, dtype=float))
    latA = np.radians(np.asarray(latA, dtype=float))
    lonB = np.radians(np.asarray(lonB, dtype=float))
    latB = np.radians(np.asarray(latB, dtype=float))
    dlon = lonA[:, None] - lonB[None, :]
    dlat = latA[:, None] - latB[None, :]
    a = np.sin(dlat/2.0)**2 + np.cos(latA)[:, None] * np.cos(latB)[None, :] * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
    return R * c
def poi_poi_od(poi,t):
    # --- precompute nearest POIs for stations using haversine via sklearn's haversine metric ---
    poi_coords = poi[['lat', 'lon']].to_numpy()
    station_coords = stations_df[['lat', 'lon']].to_numpy()
    
    # use scikit-learn NearestNeighbors with haversine (expects radians)
    nbrs = NearestNeighbors(n_neighbors=K_NEIGHBORS, metric='haversine')
    nbrs.fit(np.radians(poi_coords))
    dists_r, idxs = nbrs.kneighbors(np.radians(station_coords))
    # convert radian distances to meters
    dists_m = dists_r * 6371000.0
    
    # --- build allocations: station_id -> dict(poi_index -> weight) ---
    allocations = {}
    for i, sid in enumerate(station_ids):
        dists = dists_m[i]
        idx = idxs[i]
        # choose close ones if any inside CLOSE_RADIUS_METERS
        mask_close = dists <= CLOSE_RADIUS_METERS
        if mask_close.any():
            chosen_idx = idx[mask_close]
            chosen_dists = dists[mask_close]
        else:
            chosen_idx = idx
            chosen_dists = dists
    
        cat_weights = poi.loc[chosen_idx, 'weight'].to_numpy(dtype=float)
        inv_dist = 1.0 / (chosen_dists + EPS)
        raw_scores = cat_weights * inv_dist
        if raw_scores.sum() <= 0:
            weights = np.ones_like(raw_scores) / len(raw_scores)
        else:
            weights = raw_scores / raw_scores.sum()
    
        allocations[int(sid)] = {int(p_idx): float(w) for p_idx, w in zip(chosen_idx.tolist(), weights.tolist())}
    
    # Save allocations (flat)
    alloc_rows = []
    for s, pdict in allocations.items():
        for pidx, w in pdict.items():
            alloc_rows.append({'station_id': int(s), 'poi_index': int(pidx), 'poi_id': int(poi.at[pidx, 'poi_id']), 'weight': float(w)})
    alloc_df = pd.DataFrame(alloc_rows)
    alloc_df.to_csv(OUTPUT_DIR / 'station_poi_allocations.csv', index=False)
    print('Saved station->POI allocation table to', OUTPUT_DIR / f'station_poi_allocations_type{t}.csv')
    
    # --- Convert predicted station OD -> POI OD per cluster (using only used POIs per cluster) ---
    poi_od_per_cluster = {}
    for c, pred_df in predicted_matrices.items():
        # ensure pred_df index/cols are ints
        pred_df = pred_df.copy()
        pred_df.index = pred_df.index.astype(int)
        pred_df.columns = pred_df.columns.astype(int)
    
        # intersect stations with allocations
        common_stations = sorted(list(set(pred_df.index.tolist()) & set(allocations.keys())))
        if len(common_stations) < 2:
            print(f'Cluster {c}: too few stations with allocations ({len(common_stations)}), skipping')
            continue
    
        # build station order and Tij matrix accordingly
        Tij = pred_df.reindex(index=common_stations, columns=common_stations).fillna(0).to_numpy(dtype=float)
    
        # build station->poi allocation matrix A (nS x nP_used)
        # find all POI indices used by these stations
        poi_indices_used = sorted({pidx for s in common_stations for pidx in allocations[s].keys()})
        if len(poi_indices_used) == 0:
            print(f'Cluster {c}: no POIs assigned to these stations, skipping')
            continue
    
        nS = len(common_stations)
        nP = len(poi_indices_used)
        A = np.zeros((nS, nP), dtype=float)
        station_to_row = {s:i for i,s in enumerate(common_stations)}
        poi_to_col = {p:i for i,p in enumerate(poi_indices_used)}
    
        for s in common_stations:
            r = station_to_row[s]
            for pidx, w in allocations[s].items():
                if pidx in poi_to_col:
                    A[r, poi_to_col[pidx]] = w
    
        # compute POI OD: P = A^T * Tij * A
        P = A.T.dot(Tij).dot(A)
    
        # build DataFrame: rows/cols labelled by poi_id (not global pois index)
        poi_ids = [int(poi.at[pidx, 'poi_id']) for pidx in poi_indices_used]
        poi_od_df = pd.DataFrame(P, index=poi_ids, columns=poi_ids)
        poi_od_per_cluster[c] = poi_od_df
        poi_od_df.to_csv(OUTPUT_DIR / f'poi_od_cluster_{c}_type{t}.csv')
        print(f'Cluster {c}: saved POI OD with shape {poi_od_df.shape} (n_pois={len(poi_ids)})')
    
    # --- aggregate across clusters (align on poi_id union) ---
    if poi_od_per_cluster:
        all_poi_ids = sorted({pid for df in poi_od_per_cluster.values() for pid in df.index.tolist()})
        agg_mat = pd.DataFrame(0.0, index=all_poi_ids, columns=all_poi_ids)
        for c, df in poi_od_per_cluster.items():
            agg_mat = agg_mat.add(df.reindex(index=all_poi_ids, columns=all_poi_ids, fill_value=0), fill_value=0)
        agg_mat.to_csv(OUTPUT_DIR / f'poi_od_aggregated_all_clusters_type{t}.csv')
        print('Saved aggregated POI OD for all clusters to', OUTPUT_DIR / f'poi_od_aggregated_all_clusters_type{t}.csv')
    else:
        print('No POI OD outputs (no clusters had usable data).')
    
    print('Done. Outputs in', OUTPUT_DIR)
poi_od_u=poi_poi_od(pois_uniform, 'uniform')
poi_od_pred=poi_poi_od(pois_pred,'pred')
poi_od_c=poi_poi_od(pois_commercial,'commercial')


Saved station->POI allocation table to E:\Uni_PGT\visualisation_outputs\poi_od_fixed\station_poi_allocations_typeuniform.csv
Cluster 0: saved POI OD with shape (238, 238) (n_pois=238)
Cluster 1: saved POI OD with shape (234, 234) (n_pois=234)
Cluster 2: saved POI OD with shape (232, 232) (n_pois=232)
Cluster 3: saved POI OD with shape (232, 232) (n_pois=232)
Saved aggregated POI OD for all clusters to E:\Uni_PGT\visualisation_outputs\poi_od_fixed\poi_od_aggregated_all_clusters_typeuniform.csv
Done. Outputs in E:\Uni_PGT\visualisation_outputs\poi_od_fixed
Saved station->POI allocation table to E:\Uni_PGT\visualisation_outputs\poi_od_fixed\station_poi_allocations_typepred.csv
Cluster 0: saved POI OD with shape (238, 238) (n_pois=238)
Cluster 1: saved POI OD with shape (234, 234) (n_pois=234)
Cluster 2: saved POI OD with shape (232, 232) (n_pois=232)
Cluster 3: saved POI OD with shape (232, 232) (n_pois=232)
Saved aggregated POI OD for all clusters to E:\Uni_PGT\visualisation_outputs\poi_

In [21]:
# poi_od_sparse_with_baseline.py
#
# Memory-efficient conversion of predicted station->station OD (per cluster)
# into POI->POI OD using sparse matrices and a "baseline-to-k-nearest" strategy
# for neglected POIs so we avoid creating a dense 33k x 33k matrix.
#
# Requirements: scipy, scikit-learn, pandas, numpy
#
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy import sparse

# ---------- USER CONFIG ----------
POI_FILE = r'E:\Uni_PGT\Express\edinburgh_pois.csv'
OUTPUT_DIR = Path(r'E:\Uni_PGT\visualisation_outputs\poi_od_sparse_baseline')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

K_NEIGHBORS = 3                # used for station->poi allocation
K_BASELINE_NEIGH = 10          # number of POIs to distribute baseline for neglected POI
CLOSE_RADIUS_METERS = 500

BASELINE_RATIO = 0.1   # baseline magnitude relative to mean observed cell
EPS = 1e-12
# ---------------------------------

# sanity: required objects
try:
    predicted_matrices
except NameError:
    raise RuntimeError("`predicted_matrices` not in memory. Run gravity step first.")
try:
    combined
except NameError:
    raise RuntimeError("`combined` not in memory. Load combined dataframe first.")

# load POIs
def poi_od_w_baseline(pois,t):
    # canonical lists and lookups
    all_poi_ids = pois['poi_id'].astype(int).tolist()
    poi_index_to_id = {i: int(pois.at[i, 'poi_id']) for i in pois.index}
    poi_id_to_index = {int(pois.at[i, 'poi_id']): i for i in pois.index}
    
    n_all = len(all_poi_ids)
    print(f"POIs loaded type {t}: {n_all} (will keep results sparse)")
    
    # build station coords as median of start/end
    lon_cols = [c for c in combined.columns if 'longitude' in c.lower()]
    lat_cols = [c for c in combined.columns if 'latitude' in c.lower()]
    start_lon = next((c for c in lon_cols if c.lower().startswith('start')), None)
    start_lat = next((c for c in lat_cols if c.lower().startswith('start')), None)
    end_lon = next((c for c in lon_cols if c.lower().startswith('end')), None)
    end_lat = next((c for c in lat_cols if c.lower().startswith('end')), None)
    coord_parts = []
    if start_lon and start_lat:
        tmp = combined.groupby('start_station_id')[[start_lon, start_lat]].median()
        tmp = tmp.rename(columns={start_lon: 'lon', start_lat: 'lat'})
        coord_parts.append(tmp)
    if end_lon and end_lat:
        tmp2 = combined.groupby('end_station_id')[[end_lon, end_lat]].median()
        tmp2 = tmp2.rename(columns={end_lon: 'lon', end_lat: 'lat'})
        coord_parts.append(tmp2)
    if not coord_parts:
        raise RuntimeError('No station lon/lat columns found in combined')
    stations_df = pd.concat(coord_parts).groupby(level=0).first()
    stations_df.index = stations_df.index.astype(int)
    
    # restrict to stations that appear in at least one predicted matrix
    pred_station_ids = set()
    for m in predicted_matrices.values():
        pred_station_ids.update([int(x) for x in m.index.astype(int).tolist()])
    stations_df = stations_df.reindex(sorted(pred_station_ids)).dropna()
    station_ids = stations_df.index.astype(int).tolist()
    print(f"Using {len(station_ids)} stations for allocation")
    
    # precompute station->nearest POIs (small K)
    poi_coords = pois[['lat','lon']].to_numpy()
    station_coords = stations_df[['lat','lon']].to_numpy()
    
    nbrs = NearestNeighbors(n_neighbors=K_NEIGHBORS, metric='haversine')
    nbrs.fit(np.radians(poi_coords))
    d_radians, idxs = nbrs.kneighbors(np.radians(station_coords))
    dists_m = d_radians * 6371000.0
    
    # build allocations mapping station_id -> list of (poi_index, weight)
    allocations = {}
    for i, sid in enumerate(station_ids):
        dists = dists_m[i]
        idx = idxs[i]
        mask_close = dists <= CLOSE_RADIUS_METERS
        if mask_close.any():
            chosen_idx = idx[mask_close]
            chosen_dists = dists[mask_close]
        else:
            chosen_idx = idx
            chosen_dists = dists
        cat_weights = pois.loc[chosen_idx, 'weight'].to_numpy(dtype=float)
        inv_dist = 1.0 / (chosen_dists + EPS)
        raw = cat_weights * inv_dist
        if raw.sum() <= 0:
            w = np.ones_like(raw) / len(raw)
        else:
            w = raw / raw.sum()
        allocations[int(sid)] = list(zip([int(p) for p in chosen_idx.tolist()], [float(x) for x in w.tolist()]))
    
    # Save allocations summary
    alloc_rows = []
    for s, lst in allocations.items():
        for pidx, w in lst:
            alloc_rows.append({'station_id': s, 'poi_index': int(pidx), 'poi_id': int(pois.at[pidx,'poi_id']), 'weight': w})
    alloc_df = pd.DataFrame(alloc_rows)
    alloc_df.to_csv(OUTPUT_DIR / f'station_poi_allocations_type{t}.csv', index=False)
    print('Saved station->POI allocations (small)')
    
    # Build POI neighbor index for baseline distribution (k baseline neighbors on POI network)
    poi_nbrs = NearestNeighbors(n_neighbors=K_BASELINE_NEIGH, metric='haversine')
    poi_nbrs.fit(np.radians(poi_coords))
    poi_d_rad, poi_idxs = poi_nbrs.kneighbors(np.radians(poi_coords))
    poi_d_m = poi_d_rad * 6371000.0
    
    # helper: function to add entries to sparse accumulator lists
    
    def append_entries(rows_list, cols_list, data_list, from_poi_ids, to_poi_ids, values_matrix):
        # from_poi_ids and to_poi_ids are lists of global poi_id labels (not dataframe indices)
        for i, pid in enumerate(from_poi_ids):
            for j, qid in enumerate(to_poi_ids):
                val = values_matrix[i, j]
                if val != 0 and not np.isnan(val):
                    rows_list.append(poi_id_to_index[pid])
                    cols_list.append(poi_id_to_index[qid])
                    data_list.append(float(val))

    # main conversion loop: produce sparse COO components per cluster
    from scipy.sparse import coo_matrix, save_npz
    poi_od_sparse_per_cluster = {}
    poi_od_sparse_per_hour_per_cluster = {}  # NEW: for per-hour matrices
    
    for c, pred_df in predicted_matrices.items():
        print(f'Processing cluster {c}...')
        pred_df = pred_df.copy()
        pred_df.index = pred_df.index.astype(int)
        pred_df.columns = pred_df.columns.astype(int)

        # Get hours in this cluster for per-hour calculation
        hrs_in_cluster = hour_cluster_df[hour_cluster_df['cluster'] == c]['hour'].tolist()
        n_hrs_in_cluster = len(hrs_in_cluster)
        print(f' Cluster {c} has {n_hrs_in_cluster} hours: {hrs_in_cluster}')

        # stations in this predicted matrix that have allocations
        common_stations = sorted(list(set(pred_df.index.tolist()) & set(allocations.keys())))
        if len(common_stations) < 2:
            print(f' Cluster {c}: too few stations with allocations, skipping')
            continue

        Tij = pred_df.reindex(index=common_stations, columns=common_stations).fillna(0).to_numpy(dtype=float)

        # build A_used (nS x nP_used) where nP_used is number of distinct POIs assigned to these stations
        poi_indices_used = sorted({pidx for s in common_stations for pidx, _ in allocations[s]})
        if len(poi_indices_used) == 0:
            print(f' Cluster {c}: no POIs used by these stations, skipping')
            continue

        station_to_row = {s:i for i,s in enumerate(common_stations)}
        poi_to_col_used = {p:i for i,p in enumerate(poi_indices_used)}
        nS = len(common_stations); nP_used = len(poi_indices_used)
        A_used = np.zeros((nS, nP_used), dtype=float)
        for s in common_stations:
            r = station_to_row[s]
            for pidx, w in allocations[s]:
                if pidx in poi_to_col_used:
                    A_used[r, poi_to_col_used[pidx]] = w

        # compute P_used (nP_used x nP_used) — this is small (few hundreds)
        P_used = A_used.T.dot(Tij).dot(A_used)
        
        # NEW: Compute per-hour matrix
        if n_hrs_in_cluster > 0:
            P_used_per_hour = P_used / n_hrs_in_cluster
        else:
            P_used_per_hour = P_used  # fallback if no hours found

        # Now map P_used into global sparse lists (TOTAL version)
        rows_total = []
        cols_total = []
        data_total = []

        # map used POI local -> global poi_id
        poi_ids_used = [int(pois.at[pidx, 'poi_id']) for pidx in poi_indices_used]

        # append P_used block into sparse lists (TOTAL)
        for i_local, global_pidx in enumerate(poi_indices_used):
            pid = int(pois.at[global_pidx, 'poi_id'])
            for j_local, global_qidx in enumerate(poi_indices_used):
                qid = int(pois.at[global_qidx, 'poi_id'])
                val = P_used[i_local, j_local]
                if val != 0 and not np.isnan(val):
                    rows_total.append(poi_id_to_index[pid])
                    cols_total.append(poi_id_to_index[qid])
                    data_total.append(float(val))

        # NEW: Create separate lists for per-hour version
        rows_per_hour = []
        cols_per_hour = []
        data_per_hour = []

        # append P_used_per_hour block into sparse lists (PER-HOUR)
        for i_local, global_pidx in enumerate(poi_indices_used):
            pid = int(pois.at[global_pidx, 'poi_id'])
            for j_local, global_qidx in enumerate(poi_indices_used):
                qid = int(pois.at[global_qidx, 'poi_id'])
                val = P_used_per_hour[i_local, j_local]
                if val != 0 and not np.isnan(val):
                    rows_per_hour.append(poi_id_to_index[pid])
                    cols_per_hour.append(poi_id_to_index[qid])
                    data_per_hour.append(float(val))

        # compute baseline per-cell scalar (mean observed cell over used block)
        observed_sum = P_used.sum()
        if observed_sum <= 0:
            mean_cell = 0.0
        else:
            mean_cell = observed_sum / (n_all)  # normalized to full universe

        mean_weight = pois['weight'].mean()

        # identify neglected POIs (those that have no entries in current sparse lists)
        used_poi_set = set(poi_ids_used)
        all_poi_set = set(all_poi_ids)
        neglected = sorted(list(all_poi_set - used_poi_set))
        print(f' Cluster {c}: used POIs={len(used_poi_set)}, neglected POIs={len(neglected)}')

        # For each neglected POI, distribute baseline to its K_BASELINE_NEIGH nearest POIs (by index in pois)
        if len(neglected) > 0 and mean_cell > 0:
            # we have precomputed poi_idxs and poi_d_m arrays (POI->neighbors)
            for pid in neglected:
                pidx = poi_id_to_index[pid]
                # neighbors indices (including itself) from poi_idxs array
                neigh_local = poi_idxs[pidx, 1:K_BASELINE_NEIGH+1]  # skip self (first neighbor)
                neigh_d = poi_d_m[pidx, 1:K_BASELINE_NEIGH+1]
                # weights: inverse distance * category weight
                neigh_weights = pois.loc[neigh_local, 'weight'].to_numpy(dtype=float)
                invd = 1.0 / (neigh_d + EPS)
                raw = neigh_weights * invd
                if raw.sum() <= 0:
                    wnorm = np.ones_like(raw) / len(raw)
                else:
                    wnorm = raw / raw.sum()
                # baseline total per neglected POI (outflow) distribute across neighbors
                baseline_total = mean_cell * BASELINE_RATIO * (pois.at[pidx, 'weight'] / mean_weight)
                
                # NEW: baseline per hour
                baseline_per_hour = baseline_total / n_hrs_in_cluster if n_hrs_in_cluster > 0 else baseline_total

                # distribute baseline_total to outgoing links pid -> neigh (TOTAL)
                for k_idx, neigh_idx in enumerate(neigh_local):
                    qid = int(pois.at[int(neigh_idx), 'poi_id'])
                    val = baseline_total * wnorm[k_idx]
                    rows_total.append(pidx)
                    cols_total.append(neigh_idx)
                    data_total.append(float(val))
                # similarly distribute baseline inflow: neighbors -> pid (TOTAL)
                for k_idx, neigh_idx in enumerate(neigh_local):
                    qid = int(pois.at[int(neigh_idx), 'poi_id'])
                    val = baseline_total * wnorm[k_idx]
                    rows_total.append(neigh_idx)
                    cols_total.append(pidx)
                    data_total.append(float(val))
                    
                # NEW: distribute baseline_per_hour (PER-HOUR version)
                for k_idx, neigh_idx in enumerate(neigh_local):
                    qid = int(pois.at[int(neigh_idx), 'poi_id'])
                    val = baseline_per_hour * wnorm[k_idx]
                    rows_per_hour.append(pidx)
                    cols_per_hour.append(neigh_idx)
                    data_per_hour.append(float(val))
                for k_idx, neigh_idx in enumerate(neigh_local):
                    qid = int(pois.at[int(neigh_idx), 'poi_id'])
                    val = baseline_per_hour * wnorm[k_idx]
                    rows_per_hour.append(neigh_idx)
                    cols_per_hour.append(pidx)
                    data_per_hour.append(float(val))

        # build sparse COO and save (TOTAL version)
        coo_total = coo_matrix((data_total, (rows_total, cols_total)), shape=(n_all, n_all))
        csr_total = coo_total.tocsr()
        save_npz(OUTPUT_DIR / f'poi_od_cluster_{c}_type_{t}.npz', csr_total)

        # also save edge-list CSV (only nonzero entries) for TOTAL
        coo_nz_total = coo_total.tocoo()
        edge_df_total = pd.DataFrame({'from_poi_index': coo_nz_total.row, 'to_poi_index': coo_nz_total.col, 'flow': coo_nz_total.data})
        edge_df_total['from_poi_id'] = edge_df_total['from_poi_index'].map(lambda x: int(pois.at[int(x),'poi_id']))
        edge_df_total['to_poi_id'] = edge_df_total['to_poi_index'].map(lambda x: int(pois.at[int(x),'poi_id']))
        edge_df_total = edge_df_total[['from_poi_id','to_poi_id','flow']]
        edge_df_total.to_csv(OUTPUT_DIR / f'poi_od_cluster_{c}_type_{t}_edgelists.csv', index=False)

        # NEW: build sparse COO and save (PER-HOUR version)
        coo_per_hour = coo_matrix((data_per_hour, (rows_per_hour, cols_per_hour)), shape=(n_all, n_all))
        csr_per_hour = coo_per_hour.tocsr()
        save_npz(OUTPUT_DIR / f'poi_od_cluster_{c}_type_{t}_per_hour.npz', csr_per_hour)

        # NEW: also save edge-list CSV for PER-HOUR
        coo_nz_per_hour = coo_per_hour.tocoo()
        edge_df_per_hour = pd.DataFrame({'from_poi_index': coo_nz_per_hour.row, 'to_poi_index': coo_nz_per_hour.col, 'flow': coo_nz_per_hour.data})
        edge_df_per_hour['from_poi_id'] = edge_df_per_hour['from_poi_index'].map(lambda x: int(pois.at[int(x),'poi_id']))
        edge_df_per_hour['to_poi_id'] = edge_df_per_hour['to_poi_index'].map(lambda x: int(pois.at[int(x),'poi_id']))
        edge_df_per_hour = edge_df_per_hour[['from_poi_id','to_poi_id','flow']]
        edge_df_per_hour.to_csv(OUTPUT_DIR / f'poi_od_cluster_{c}_type_{t}_per_hour_edgelists.csv', index=False)

        poi_od_sparse_per_cluster[c] = {'sparse': csr_total, 'edge_csv': OUTPUT_DIR / f'poi_od_cluster_{c}_type_{t}_edgelist.csv'}
        poi_od_sparse_per_hour_per_cluster[c] = {'sparse': csr_per_hour, 'edge_csv': OUTPUT_DIR / f'poi_od_cluster_{c}_type_{t}_per_hour_edgelist.csv'}  # NEW
        
        print(f' Cluster {c}: saved sparse POI OD (nnz={csr_total.nnz}) and per-hour POI OD (nnz={csr_per_hour.nnz})')

    # aggregate across clusters (sum sparse matrices) - TOTAL version
    if poi_od_sparse_per_cluster:
        first = True
        agg_csr_total = None
        for c, info in poi_od_sparse_per_cluster.items():
            mat = info['sparse']
            if first:
                agg_csr_total = mat.copy()
                first = False
            else:
                agg_csr_total = agg_csr_total + mat
        # save aggregated TOTAL
        save_npz(OUTPUT_DIR / f'poi_od_aggregated_all_clusters_sparse_type_{t}.npz', agg_csr_total)
        # also save aggregated edge list for TOTAL
        coo_agg_total = agg_csr_total.tocoo()
        edge_agg_total = pd.DataFrame({'from_idx': coo_agg_total.row, 'to_idx': coo_agg_total.col, 'flow': coo_agg_total.data})
        edge_agg_total['from_poi_id'] = edge_agg_total['from_idx'].map(lambda x: int(pois.at[int(x),'poi_id']))
        edge_agg_total['to_poi_id'] = edge_agg_total['to_idx'].map(lambda x: int(pois.at[int(x),'poi_id']))
        edge_agg_total[['from_poi_id','to_poi_id','flow']].to_csv(OUTPUT_DIR / f'poi_od_aggregated_all_clusters_edgelists_type_{t}.csv', index=False)
        
        # NEW: aggregate across clusters for PER-HOUR version
        first = True
        agg_csr_per_hour = None
        for c, info in poi_od_sparse_per_hour_per_cluster.items():
            mat = info['sparse']
            if first:
                agg_csr_per_hour = mat.copy()
                first = False
            else:
                agg_csr_per_hour = agg_csr_per_hour + mat
        # save aggregated PER-HOUR
        save_npz(OUTPUT_DIR / f'poi_od_aggregated_all_clusters_sparse_type_{t}_per_hour.npz', agg_csr_per_hour)
        # also save aggregated edge list for PER-HOUR
        coo_agg_per_hour = agg_csr_per_hour.tocoo()
        edge_agg_per_hour = pd.DataFrame({'from_idx': coo_agg_per_hour.row, 'to_idx': coo_agg_per_hour.col, 'flow': coo_agg_per_hour.data})
        edge_agg_per_hour['from_poi_id'] = edge_agg_per_hour['from_idx'].map(lambda x: int(pois.at[int(x),'poi_id']))
        edge_agg_per_hour['to_poi_id'] = edge_agg_per_hour['to_idx'].map(lambda x: int(pois.at[int(x),'poi_id']))
        edge_agg_per_hour[['from_poi_id','to_poi_id','flow']].to_csv(OUTPUT_DIR / f'poi_od_aggregated_all_clusters_edgelists_type_{t}_per_hour.csv', index=False)
        
        print('Saved aggregated sparse POI OD and per-hour POI OD with edge lists')

    print('Done. Outputs (sparse .npz + edgelists) in', OUTPUT_DIR)
    
poi_od_w_baseline(pois_uniform,'uniform')
poi_od_w_baseline(pois_pred,'pred')
poi_od_w_baseline(pois_commercial,'commercial')


POIs loaded type uniform: 885 (will keep results sparse)
Using 194 stations for allocation
Saved station->POI allocations (small)
Processing cluster 0...
 Cluster 0 has 5 hours: [11, 12, 13, 14, 15]
 Cluster 0: used POIs=238, neglected POIs=647
 Cluster 0: saved sparse POI OD (nnz=64154) and per-hour POI OD (nnz=64154)
Processing cluster 1...
 Cluster 1 has 2 hours: [16, 17]
 Cluster 1: used POIs=234, neglected POIs=651
 Cluster 1: saved sparse POI OD (nnz=62288) and per-hour POI OD (nnz=62288)
Processing cluster 2...
 Cluster 2 has 6 hours: [18, 19, 20, 21, 22, 23]
 Cluster 2: used POIs=232, neglected POIs=653
 Cluster 2: saved sparse POI OD (nnz=61366) and per-hour POI OD (nnz=61366)
Processing cluster 3...
 Cluster 3 has 11 hours: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
 Cluster 3: used POIs=232, neglected POIs=653
 Cluster 3: saved sparse POI OD (nnz=61366) and per-hour POI OD (nnz=61366)
Saved aggregated sparse POI OD and per-hour POI OD with edge lists
Done. Outputs (sparse .npz + edg

In [22]:
# kmeans_candidates_no_snap.py
# Demand-weighted KMeans (NO snapping to POIs) to produce P candidate stations
# Inputs:
#  - POI_FILE: CSV with columns ['poi_id'(optional),'lat','lon',...]
#  - POI_EDGELIST: combined POI edgelist CSV with columns [from_poi_id,to_poi_id,flow]
# Outputs saved to OUTPUT_DIR:
#  - candidate_stations_P{P}_kmeans_no_snap.csv (centroid lon/lat + demand stats)
#  - candidate_eval_P{P}_kmeans_no_snap.csv

from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# --------- USER CONFIG ----------

POI_EDGELIST =Path( r'E:\Uni_PGT\visualisation_outputs\poi_od_sparse_baseline')
OUTPUT_DIR = Path(r'E:\Uni_PGT\visualisation_outputs\candidate_stations\candidate_stations_with_existing')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

EXISTING_STATIONS_FILE = r'E:\Uni_PGT\station_data.csv'  # New: path to existing stations

P = 150                          # number of candidate centroids to produce
KMEANS_RANDOM_STATE = 42
KM_INIT = 'k-means++'           # 'k-means++' or 'random'
COVERAGE_RADIUS_METERS = 800.0  # evaluation radius for coverage (walk radius)
SNAP_DISTANCE_METERS = 40.0     # distance threshold for snapping to existing stations
# --------------------------------

# Utility: small haversine helpers (meters)
def haversine_matrix(lonsA, latsA, lonsB, latsB):
    R = 6371000.0
    lonA = np.radians(np.asarray(lonsA, dtype=float))
    latA = np.radians(np.asarray(latsA, dtype=float))
    lonB = np.radians(np.asarray(lonsB, dtype=float))
    latB = np.radians(np.asarray(latsB, dtype=float))
    dlon = lonA[:, None] - lonB[None, :]
    dlat = latA[:, None] - latB[None, :]
    a = np.sin(dlat/2.0)**2 + np.cos(latA)[:, None] * np.cos(latB)[None, :] * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
    return R * c

# --- Load existing stations ---
print('Loading existing stations...')
try:
    existing_stations = pd.read_csv(EXISTING_STATIONS_FILE )
    # Try to detect station ID and coordinate columns
    station_id_col = None
    lat_col = None
    lon_col = None
    
    # Common column name patterns
    for col in existing_stations.columns:
        col_lower = col.lower()
        if 'id' in col_lower and station_id_col is None:
            station_id_col = col
        elif 'lat' in col_lower and lat_col is None:
            lat_col = col
        elif 'lon' in col_lower and lon_col is None:
            lon_col = col
    
    # If not found, use first columns
    if station_id_col is None:
        station_id_col = existing_stations.columns[0]
    if lat_col is None:
        lat_col = existing_stations.columns[1]
    if lon_col is None:
        lon_col = existing_stations.columns[2]
    
    print(f"Using columns - ID: {station_id_col}, Lat: {lat_col}, Lon: {lon_col}")
    
    existing_stations = existing_stations[[station_id_col, lat_col, lon_col]].copy()
    existing_stations.columns = ['station_id', 'lat', 'lon']
    existing_stations = existing_stations.dropna(subset=['lat', 'lon'])
    
    print(f"Loaded {len(existing_stations)} existing stations")
    
except Exception as e:
    print(f"Error loading existing stations: {e}")
    existing_stations = pd.DataFrame(columns=['station_id', 'lat', 'lon'])

# --- load POIs and compute demand per POI from edgelist ---
def candidate_gen(pois,t,p):   
    if 'poi_id' not in pois.columns:
        pois['poi_id'] = pois.index.astype(int)
    pois['poi_id'] = pois['poi_id'].astype(int)
    
    print(f'Loading POI type {t} edgelist to compute demand per POI ...')
    edges = pd.read_csv(POI_EDGELIST/ f'poi_od_aggregated_all_clusters_edgelists_type_{t}.csv')
    # detect flow column
    cols_lower = {c.lower(): c for c in edges.columns}
    flow_col = None
    for candidate in ('flow', 'demand', 'count', 'weight'):
        if candidate in cols_lower:
            flow_col = cols_lower[candidate]
            break
    if flow_col is None:
        numeric_cols = [c for c in edges.columns if np.issubdtype(edges[c].dtype, np.number)]
        if numeric_cols:
            flow_col = numeric_cols[-1]
        else:
            raise RuntimeError('Could not detect numeric flow column in edgelist.')
    print('Flow column detected:', flow_col)
    
    inflow = edges.groupby('to_poi_id')[flow_col].sum()
    outflow = edges.groupby('from_poi_id')[flow_col].sum()
    all_ids = sorted(set(inflow.index.tolist()) | set(outflow.index.tolist()) | set(pois['poi_id'].tolist()))
    
    # demand = inflow + outflow (symmetric importance for being origin/destination)
    demand_series = pd.Series(0.0, index=all_ids, dtype=float)
    if not inflow.empty:
        demand_series.loc[inflow.index] += inflow
    if not outflow.empty:
        demand_series.loc[outflow.index] += outflow
    
    poi_demand = pois.set_index('poi_id').reindex(all_ids).copy()
    poi_demand['demand'] = demand_series.reindex(poi_demand.index).fillna(0.0)
    
    # Remove POIs with missing coordinates
    poi_demand = poi_demand.dropna(subset=['lat', 'lon'])
    
    poi_ids = poi_demand.index.astype(int).tolist()
    lats = poi_demand['lat'].to_numpy(dtype=float)
    lons = poi_demand['lon'].to_numpy(dtype=float)
    weights = poi_demand['demand'].to_numpy(dtype=float)
    
    total_demand = weights.sum()
    print(f'POIs loaded: {len(poi_ids)} POIs, total demand = {total_demand:.2f}')
    
    # avoid zero-weight issues
    weights_safe = weights.copy()
    weights_safe[weights_safe <= 0] = 1e-6
    
    # Project lon/lat to a simple local metric space (equirectangular approx) for KMeans
    mean_lat = np.mean(lats)
    xm = (lons - np.mean(lons)) * (111320 * np.cos(np.radians(mean_lat)))
    ym = (lats - np.mean(lats)) * 110540
    X_m = np.column_stack([xm, ym])
    
    # Run weighted KMeans (sample_weight)
    print('Running demand-weighted KMeans (no snapping) ...')
    km = KMeans(n_clusters=p, init=KM_INIT, random_state=KMEANS_RANDOM_STATE, n_init=10)
    km.fit(X_m, sample_weight=weights_safe)
    centers_m = km.cluster_centers_
    
    # Map centers back to lon/lat approx
    centers_lon = (centers_m[:,0] / (111320 * np.cos(np.radians(mean_lat)))) + np.mean(lons)
    centers_lat = (centers_m[:,1] / 110540) + np.mean(lats)
    
    # Create initial candidates DataFrame
    candidates = pd.DataFrame({
        'candidate_id': range(len(centers_lon)),
        'centroid_lon': centers_lon,
        'centroid_lat': centers_lat,
        'is_existing_station': False,  # Initialize as new stations
        'original_candidate_id': range(len(centers_lon))  # Track original KMeans ID
    })
    
    # --- Snap candidates to existing stations if within 20 meters ---
    if len(existing_stations) > 0:
        print(f"Snapping candidates to existing stations within {SNAP_DISTANCE_METERS} meters...")
        
        # Calculate distance matrix between candidates and existing stations
        candidate_coords = candidates[['centroid_lat', 'centroid_lon']].values
        existing_coords = existing_stations[['lat', 'lon']].values
        
        # Use haversine distance
        distances = haversine_matrix(
            candidate_coords[:, 1], candidate_coords[:, 0],  # lon, lat
            existing_coords[:, 1], existing_coords[:, 0]     # lon, lat
        )
        
        # Find nearest existing station for each candidate
        min_distances = distances.min(axis=1)
        nearest_station_indices = distances.argmin(axis=1)
        
        snapped_count = 0
        for i in range(len(candidates)):
            if min_distances[i] <= SNAP_DISTANCE_METERS:
                # Snap to existing station
                station_idx = nearest_station_indices[i]
                candidates.at[i, 'centroid_lat'] = existing_stations.iloc[station_idx]['lat']
                candidates.at[i, 'centroid_lon'] = existing_stations.iloc[station_idx]['lon']
                candidates.at[i, 'is_existing_station'] = True
                candidates.at[i, 'snapped_station_id'] = existing_stations.iloc[station_idx]['station_id']
                snapped_count += 1
        
        print(f"Snapped {snapped_count} candidates to existing stations")
    
    # --- Add any missing existing stations ---
    if len(existing_stations) > 0:
        print("Adding any missing existing stations to candidate set...")
        
        # Check which existing stations are not already in candidates
        existing_coords = existing_stations[['lat', 'lon']].values
        candidate_coords = candidates[['centroid_lat', 'centroid_lon']].values
        
        # Calculate distances between existing stations and all candidates
        distances = haversine_matrix(
            existing_coords[:, 1], existing_coords[:, 0],  # lon, lat
            candidate_coords[:, 1], candidate_coords[:, 0] # lon, lat
        )
        
        # Find existing stations that are not close to any candidate
        min_distances_to_candidates = distances.min(axis=1)
        missing_existing_mask = min_distances_to_candidates > SNAP_DISTANCE_METERS
        
        missing_existing_count = missing_existing_mask.sum()
        if missing_existing_count > 0:
            print(f"Adding {missing_existing_count} missing existing stations")
            
            # Add missing existing stations
            missing_stations = existing_stations[missing_existing_mask].copy()
            new_candidates = pd.DataFrame({
                'candidate_id': range(len(candidates), len(candidates) + len(missing_stations)),
                'centroid_lon': missing_stations['lon'],
                'centroid_lat': missing_stations['lat'],
                'is_existing_station': True,
                'snapped_station_id': missing_stations['station_id'],
                'original_candidate_id': -1  # Mark as not from KMeans
            })
            
            candidates = pd.concat([candidates, new_candidates], ignore_index=True)
    
    
    print(f"Final candidate set: {len(candidates)} stations ({candidates['is_existing_station'].sum()} existing, {len(candidates) - candidates['is_existing_station'].sum()} new)")
    
    # Evaluate coverage: distance from each POI to nearest candidate
    candidate_coords_final = candidates[['centroid_lat', 'centroid_lon']].values
    D = haversine_matrix(lons, lats, candidate_coords_final[:, 1], candidate_coords_final[:, 0])
    min_dist = D.min(axis=1)
    within = (min_dist <= COVERAGE_RADIUS_METERS)
    covered_demand = weights[within].sum()
    coverage_fraction = covered_demand / (weights.sum() + 1e-12)
    avg_weighted_dist = np.sum(min_dist * weights) / (weights.sum() + 1e-12)
    median_dist = np.median(min_dist)
    
    eval_metrics = {
        'n_candidates': len(candidates),
        'n_existing_stations': int(candidates['is_existing_station'].sum()),
        'n_new_stations': len(candidates) - candidates['is_existing_station'].sum(),
        'coverage_fraction_demand': float(coverage_fraction),
        'avg_weighted_distance_m': float(avg_weighted_dist),
        'median_distance_m': float(median_dist),
        'num_pois_total': len(poi_ids),
        'total_demand': float(weights.sum())
    }
    
    # Save outputs
    cand_out = OUTPUT_DIR / f'candidate_stations_P{p}_type_{t}.csv'
    candidates.to_csv(cand_out, index=False)
    pd.Series(eval_metrics).to_frame('value').to_csv(OUTPUT_DIR / f'candidate_eval_P{p}_type_{t}.csv')
    
    print('Saved candidates to:', cand_out)
    print('Saved evaluation to:', OUTPUT_DIR / f'candidate_eval_P{p}_type_{t}.csv')
    print('Done.')
candidate_gen(pois_uniform,'uniform',50)
candidate_gen(pois_uniform,'uniform',85)
candidate_gen(pois_uniform,'uniform',100)
candidate_gen(pois_uniform,'uniform',150)
candidate_gen(pois_uniform,'uniform',200)
candidate_gen(pois_uniform,'uniform',250)
candidate_gen(pois_uniform,'uniform',300)
candidate_gen(pois_pred,'pred',50)
candidate_gen(pois_pred,'pred',85)
candidate_gen(pois_pred,'pred',100)
candidate_gen(pois_pred,'pred',150)
candidate_gen(pois_pred,'pred',200)
candidate_gen(pois_pred,'pred',250)
candidate_gen(pois_pred,'pred',300)
candidate_gen(pois_commercial,'commercial',50)
candidate_gen(pois_commercial,'commercial',85)
candidate_gen(pois_commercial,'commercial',100)
candidate_gen(pois_commercial,'commercial',150)
candidate_gen(pois_commercial,'commercial',200)
candidate_gen(pois_commercial,'commercial',250)
candidate_gen(pois_commercial,'commercial',300)

Loading existing stations...
Using columns - ID: station_id, Lat: lat, Lon: lon
Loaded 85 existing stations
Loading POI type uniform edgelist to compute demand per POI ...
Flow column detected: flow
POIs loaded: 885 POIs, total demand = 982.99
Running demand-weighted KMeans (no snapping) ...
Snapping candidates to existing stations within 40.0 meters...
Snapped 1 candidates to existing stations
Adding any missing existing stations to candidate set...
Adding 84 missing existing stations
Final candidate set: 134 stations (85 existing, 49 new)
Saved candidates to: E:\Uni_PGT\visualisation_outputs\candidate_stations\candidate_stations_with_existing\candidate_stations_P50_type_uniform.csv
Saved evaluation to: E:\Uni_PGT\visualisation_outputs\candidate_stations\candidate_stations_with_existing\candidate_eval_P50_type_uniform.csv
Done.
Loading POI type uniform edgelist to compute demand per POI ...
Flow column detected: flow
POIs loaded: 885 POIs, total demand = 982.99
Running demand-weighted

In [25]:
# Calculate POI demand as max(inflow, outflow) for both normal and per-hour versions
from pathlib import Path
import pandas as pd
import numpy as np

def calculate_poi_demand(pois, t, c):
    """
    Calculate demand for each POI as max(sum(inflow), sum(outflow))
    for both normal and per-hour POI OD matrices PER CLUSTER
    """
    # Paths to the edge list files
    POI_EDGELIST_DIR = Path(r'E:\Uni_PGT\visualisation_outputs\poi_od_sparse_baseline')
    
    # Files for normal (total) demand - now cluster-specific
    normal_edges_file = POI_EDGELIST_DIR / f'poi_od_cluster_{c}_type_{t}_edgelists.csv'
    
    # Files for per-hour demand - now cluster-specific  
    per_hour_edges_file = POI_EDGELIST_DIR / f'poi_od_cluster_{c}_type_{t}_per_hour_edgelists.csv'
    
    # Output directory
    OUTPUT_DIR = Path(r'E:\Uni_PGT\visualisation_outputs\poi_demand_analysis')
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    # Process normal (total) demand
    print(f"Processing normal demand for type {t} cluster {c}...")
    if normal_edges_file.exists():
        edges = pd.read_csv(normal_edges_file)
        
        # Calculate inflow (sum of flows where POI is destination)
        inflow = edges.groupby('to_poi_id')['flow'].sum().reset_index()
        inflow.columns = ['poi_id', 'inflow']
        
        # Calculate outflow (sum of flows where POI is origin)
        outflow = edges.groupby('from_poi_id')['flow'].sum().reset_index()
        outflow.columns = ['poi_id', 'outflow']
        
        # Merge inflow and outflow
        demand_df = pd.merge(inflow, outflow, on='poi_id', how='outer').fillna(0)
        
        # Calculate demand as max(inflow, outflow)
        demand_df['demand'] = demand_df[['inflow', 'outflow']].max(axis=1)
        
        # Add POI category information
        poi_info = pois[['poi_id', 'category', 'lat', 'lon']].copy()
        demand_df = pd.merge(demand_df, poi_info, on='poi_id', how='left')
        
        # Save normal demand with cluster identifier
        demand_df.to_csv(OUTPUT_DIR / f'poi_demand_normal_type_{t}_cluster{c}.csv', index=False)
        print(f"Saved normal demand for {len(demand_df)} POIs in cluster {c}")
        
        # Summary by category for this cluster
        category_summary = demand_df.groupby('category').agg({
            'poi_id': 'count',
            'inflow': 'sum',
            'outflow': 'sum', 
            'demand': 'sum'
        }).rename(columns={'poi_id': 'poi_count'})
        category_summary.to_csv(OUTPUT_DIR / f'poi_demand_normal_category_summary_type_{t}_cluster{c}.csv')
        print(f"Saved category summary for normal demand in cluster {c}")
        
    else:
        print(f"Warning: Normal edges file not found: {normal_edges_file}")
        demand_df = pd.DataFrame()
    
    # Process per-hour demand
    print(f"Processing per-hour demand for type {t} cluster {c}...")
    if per_hour_edges_file.exists():
        edges_per_hour = pd.read_csv(per_hour_edges_file)
        
        # Calculate inflow (sum of flows where POI is destination)
        inflow_ph = edges_per_hour.groupby('to_poi_id')['flow'].sum().reset_index()
        inflow_ph.columns = ['poi_id', 'inflow_per_hour']
        
        # Calculate outflow (sum of flows where POI is origin)
        outflow_ph = edges_per_hour.groupby('from_poi_id')['flow'].sum().reset_index()
        outflow_ph.columns = ['poi_id', 'outflow_per_hour']
        
        # Merge inflow and outflow
        demand_ph_df = pd.merge(inflow_ph, outflow_ph, on='poi_id', how='outer').fillna(0)
        
        # Calculate demand as max(inflow, outflow)
        demand_ph_df['demand_per_hour'] = demand_ph_df[['inflow_per_hour', 'outflow_per_hour']].sum(axis=1)
        
        # Add POI category information
        poi_info = pois[['poi_id', 'category', 'lat', 'lon']].copy()
        demand_ph_df = pd.merge(demand_ph_df, poi_info, on='poi_id', how='left')
        
        # Save per-hour demand with cluster identifier
        demand_ph_df.to_csv(OUTPUT_DIR / f'poi_demand_per_hour_type_{t}_cluster{c}.csv', index=False)
        print(f"Saved per-hour demand for {len(demand_ph_df)} POIs in cluster {c}")
        
        # Summary by category for per-hour in this cluster
        category_summary_ph = demand_ph_df.groupby('category').agg({
            'poi_id': 'count',
            'inflow_per_hour': 'sum',
            'outflow_per_hour': 'sum',
            'demand_per_hour': 'sum'
        }).rename(columns={'poi_id': 'poi_count'})
        category_summary_ph.to_csv(OUTPUT_DIR / f'poi_demand_per_hour_category_summary_type_{t}_cluster{c}.csv')
        print(f"Saved category summary for per-hour demand in cluster {c}")
        
        # Combine both normal and per-hour demands for comparison for this cluster
        if not demand_df.empty:
            combined_demand = pd.merge(
                demand_df[['poi_id', 'category', 'inflow', 'outflow', 'demand']],
                demand_ph_df[['poi_id', 'inflow_per_hour', 'outflow_per_hour', 'demand_per_hour']],
                on='poi_id', 
                how='outer'
            ).fillna(0)
            
            # Calculate ratio of per-hour to normal demand
            combined_demand['demand_ratio'] = combined_demand['demand_per_hour'] / (combined_demand['demand'] + 1e-12)
            
            combined_demand.to_csv(OUTPUT_DIR / f'poi_demand_combined_type_{t}_cluster{c}.csv', index=False)
            print(f"Saved combined demand analysis for cluster {c}")
            
    else:
        print(f"Warning: Per-hour edges file not found: {per_hour_edges_file}")
    
    print(f"Completed demand analysis for type {t} cluster {c}")
    return OUTPUT_DIR

def create_summary_across_types_and_clusters(clusters):
    """Create a summary comparing demand across all three weight types and all clusters"""
    OUTPUT_DIR = Path(r'E:\Uni_PGT\visualisation_outputs\poi_demand_analysis')
    
    summary_data = []
    for t in ['uniform', 'pred', 'commercial']:
        for c in clusters:
            # Try to load normal demand files for this cluster and type
            normal_file = OUTPUT_DIR / f'poi_demand_normal_type_{t}_cluster{c}.csv'
            per_hour_file = OUTPUT_DIR / f'poi_demand_per_hour_type_{t}_cluster{c}.csv'
            
            if normal_file.exists():
                df_normal = pd.read_csv(normal_file)
                total_demand_normal = df_normal['demand'].sum()
                avg_demand_normal = df_normal['demand'].mean()
                max_demand_normal = df_normal['demand'].max()
                n_pois_normal = len(df_normal)
            else:
                total_demand_normal = avg_demand_normal = max_demand_normal = 0
                n_pois_normal = 0
                
            if per_hour_file.exists():
                df_per_hour = pd.read_csv(per_hour_file)
                total_demand_ph = df_per_hour['demand_per_hour'].sum()
                avg_demand_ph = df_per_hour['demand_per_hour'].mean()
                max_demand_ph = df_per_hour['demand_per_hour'].max()
                n_pois_ph = len(df_per_hour)
            else:
                total_demand_ph = avg_demand_ph = max_demand_ph = 0
                n_pois_ph = 0
                
            summary_data.append({
                'type': t,
                'cluster': c,
                'total_demand_normal': total_demand_normal,
                'avg_demand_normal': avg_demand_normal,
                'max_demand_normal': max_demand_normal,
                'total_demand_per_hour': total_demand_ph,
                'avg_demand_per_hour': avg_demand_ph,
                'max_demand_per_hour': max_demand_ph,
                'n_pois_normal': n_pois_normal,
                'n_pois_per_hour': n_pois_ph
            })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(OUTPUT_DIR / 'demand_summary_across_types_and_clusters.csv', index=False)
    print("Saved cross-type and cross-cluster summary")
    return summary_df

# Main execution
print("Starting POI demand analysis per cluster...")

# Define the clusters we have (based on your clustering results)
clusters = [0, 1, 2, 3]

# Run for all three weight types and all clusters
output_dirs = {}
for t, pois_data in [('uniform', pois_uniform), ('pred', pois_pred), ('commercial', pois_commercial)]:
    print(f"\n{'='*50}")
    print(f"Analyzing demand for type: {t}")
    print(f"{'='*50}")
    
    for c in clusters:
        print(f"\n--- Processing Cluster {c} ---")
        output_dirs[f"{t}_cluster{c}"] = calculate_poi_demand(pois_data, t, c)

print(f"\n{'='*60}")
print("ALL CLUSTER DEMAND ANALYSES COMPLETED!")
print("Results saved in:", output_dirs[list(output_dirs.keys())[0]] if output_dirs else "No output directories")
print(f"{'='*60}")

# Create comprehensive summary across all types and clusters
print("\nCreating comprehensive summary across all types and clusters...")
summary_df = create_summary_across_types_and_clusters(clusters)
print("\nSummary across all types and clusters:")
print(summary_df)

# Additional: Create cluster-specific summaries
print("\nCreating individual cluster summaries...")
for c in clusters:
    cluster_summary = summary_df[summary_df['cluster'] == c]
    print(f"\nCluster {c} Summary:")
    print(cluster_summary[['type', 'total_demand_normal', 'total_demand_per_hour', 'n_pois_normal']])

Starting POI demand analysis per cluster...

Analyzing demand for type: uniform

--- Processing Cluster 0 ---
Processing normal demand for type uniform cluster 0...
Saved normal demand for 885 POIs in cluster 0
Saved category summary for normal demand in cluster 0
Processing per-hour demand for type uniform cluster 0...
Saved per-hour demand for 885 POIs in cluster 0
Saved category summary for per-hour demand in cluster 0
Saved combined demand analysis for cluster 0
Completed demand analysis for type uniform cluster 0

--- Processing Cluster 1 ---
Processing normal demand for type uniform cluster 1...
Saved normal demand for 885 POIs in cluster 1
Saved category summary for normal demand in cluster 1
Processing per-hour demand for type uniform cluster 1...
Saved per-hour demand for 885 POIs in cluster 1
Saved category summary for per-hour demand in cluster 1
Saved combined demand analysis for cluster 1
Completed demand analysis for type uniform cluster 1

--- Processing Cluster 2 ---
Pr